In [4]:
import tensorflow as tf
import numpy as np

# Example dummy model (replace with your trained model)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train with dummy data (replace with real dataset)
x_train = np.random.rand(1000, 28, 28, 1).astype(np.float32)
y_train = np.random.randint(0, 10, 1000)

model.fit(x_train, y_train, epochs=50)

# Save model
model.export("saved_model")


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0878 - loss: 2.3422
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1258 - loss: 2.2892
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.1574 - loss: 2.2537
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.2257 - loss: 2.1999
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.3515 - loss: 2.1216
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.4086 - loss: 2.0489
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5327 - loss: 1.9360
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6285 - loss: 1.8362
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6638 - loss: 1.7200
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7740 - loss: 1.5744
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7898 - loss: 1.4765
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy:

In [5]:
# Load SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")

# Enable full integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative dataset generator
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 28, 28, 1).astype(np.float32)
        yield [data]

converter.representative_dataset = representative_dataset

# Force full INT8
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]

converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert
tflite_model = converter.convert()

# Save INT8 model
with open("model_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("INT8 model generated successfully.")


INT8 model generated successfully.


In [6]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="model_int8.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input dtype:", input_details[0]['dtype'])
print("Output dtype:", output_details[0]['dtype'])


Input dtype: <class 'numpy.int8'>
Output dtype: <class 'numpy.int8'>


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
